| Name | Type (Shape) | Description |
| --- | --- | --- |
| time | array (Nt,) | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (NX, NY) | X-meshgrid of velocity data|
| Y | array (NX, NY) | Y-meshgrid of velocity data|
| Interpolant_u | Interpolant object | Interpolant object for $ u(\mathbf{x}, t)  $ |
| Interpolant_v | Interpolant object | Interpolant object for $ v(\mathbf{x}, t)  $ |
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in time|
| defined_domain | array (NX, NY) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| time_data | array(1,NT) | time of velocity data |
| delta | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| Fmap | array (Nt, 2, ) | integrated trajectory |
| dFdt | array (Nt, 2, ) | velocity along trajectory |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add utils folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# Import package for progress bar
from tqdm.notebook import tqdm

# Import function which computes RK4 step
from ipynb.fs.defs.RK4_step import RK4_step

In [3]:
def integration_dFdt(time, x, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data):
    
    # reshape x
    x = x.reshape(2, -1)

    # Initialize arrays for flow map and derivative of flow map
    Fmap = np.zeros((len(time), 2, x.shape[1]))
    dFdt = np.zeros((len(time)-1, 2, x.shape[1]))
    
    # Step-size
    dt = time[1]-time[0]
    
    counter = 0

    # initial conditions
    Fmap[counter,:,:] = x
    
    # Runge Kutta 4th order integration with fixed step size dt
    for t in time[:-1]:
        
        Fmap[counter+1,:, :], dFdt[counter,:,:] = RK4_step(t, Fmap[counter,:, :], dt, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)
        
        # check if periodic in x
        if periodic[0]:
        
            Fmap[counter+1,0,:] = (Fmap[counter+1, 0,:]-X[0,0])%(X[0, -1]-X[0, 0])+X[0,0]
    
        # check if periodic in y
        if periodic[1]:
        
            Fmap[counter+1,1,:] = (Fmap[counter+1, 1, :]-Y[0,0])%(Y[-1, 0]-Y[0, 0])+Y[0,0]
    
        counter += 1
    
    return Fmap, dFdt